In [28]:
import os 
import dotenv

dotenv.load_dotenv()
os.environ["SERPAPI_API_KEY"] = os.getenv("SERPAPI_API_KEY") 

In [54]:
from langchain.agents import load_tools
from langchain_huggingface import ChatHuggingFace,HuggingFaceEndpoint

llm = HuggingFaceEndpoint(
    repo_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    task = 'text-generation',
    timeout = 1000
)

model = ChatHuggingFace(llm = llm)
toolbox = load_tools(tool_names = ['serpapi'],llm = model )

In [55]:
import requests 
from datetime import datetime 
from langchain.agents import tool

@tool
def get_location_from_ip():
    ''' Get the geographical location based on the IP address'''

    try:
        response = requests.get("https://ipinfo.io/json")
        data = response.json()

        if 'loc' in data:
            latitude , longitude = data['loc'].split(',')
            data = (
                f"Latitude: {latitude}",
                f"Longitude: {longitude},\n"
                f"City: {data.get('city', 'N/A')},\n"
                f"Country: {data.get('country', 'N/A')}"
            ) 
            return data
        else:
           return "Location could not be determined...."
    except Exception as e:
        return f"Error occured : {e}"
    

@tool     
def get_current_datetime() -> str:
    ''' Get the current data and time'''
    return datetime.now().strftime("%d-%m-%Y %H:%M:%S")

In [56]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
    ("system", "you're a helpful assistant"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}")
    ]
)

tools = toolbox+[get_current_datetime , get_location_from_ip]


In [57]:
from langchain.agents import create_tool_calling_agent,AgentExecutor

agent = create_tool_calling_agent(
    llm = model,
    tools = tools , 
    prompt = prompt 
)

agent_executor = AgentExecutor(
    agent = agent ,
    tools = tools ,
    verbose = True
) 

In [58]:
out = agent_executor.invoke({
    "input": (
        "I have a few questions, what is the date and time right now? "
        "How is the weather where I am? Please give me degrees in Celsius"
    )
})




> Entering new AgentExecutor chain...

Invoking: `Search` with `GoogleNews API search query`


['Our Google News API allows you to scrape results from the Google News search page. The API is accessed through the following endpoint: /search?engine=google_ ...', 'Custom Search API is one of the ways to implement Programmable Search Engine on your site. The results are returned using the Programmable Search Engine ...', 'Query your search traffic data with filters and parameters that you define. The method returns zero or more rows grouped by the row keys (dimensions) that you ...', 'A powerful REST News API to search for current and historical articles and get trending news in over 22 languages across 30 countries from 60000+ sources.', 'Consider using the Google News RSS feed. It is already formatted in a easy to parse format. You can perform a search using the following format.', 'The Google News API empowers developers to integrate live news feeds and search functionalities into the

HfHubHTTPError: 503 Server Error: Service Temporarily Unavailable for url: https://router.huggingface.co/hf-inference/models/TinyLlama/TinyLlama-1.1B-Chat-v1.0/v1/chat/completions

In [59]:
out = agent_executor.invoke({
    "input": (
        "I have a few questions, what is the date and time right now? "
        "How is the weather where I am? Please give me degrees in Celsius"
    )
})




> Entering new AgentExecutor chain...

Invoking: `Search` with `GoogleNews API search query`


['Our Google News API allows you to scrape results from the Google News search page. The API is accessed through the following endpoint: /search?engine=google_ ...', 'Custom Search API is one of the ways to implement Programmable Search Engine on your site. The results are returned using the Programmable Search Engine ...', 'Query your search traffic data with filters and parameters that you define. The method returns zero or more rows grouped by the row keys (dimensions) that you ...', 'A powerful REST News API to search for current and historical articles and get trending news in over 22 languages across 30 countries from 60000+ sources.', 'Consider using the Google News RSS feed. It is already formatted in a easy to parse format. You can perform a search using the following format.', 'The Google News API empowers developers to integrate live news feeds and search functionalities into the

ValueError: Got error from SerpAPI: Missing query `q` parameter.